In [1]:
import sys 
from pathlib import Path
import os 

In [2]:
parent_folder = Path(os.getcwd()).resolve().parent
sys.path.insert(0, str(parent_folder))
print('parent_folder ', parent_folder)

parent_folder  /home/jin/gpt2_from_scratch


In [3]:
from gpt_model import GPT2
from gpt2_config.config import  GPT2Config

parent_folder  /home/jin/gpt2_from_scratch/gpt2


In [4]:
config = GPT2Config()
model = GPT2(cfg=config)

vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>
vocab size : <gpt2_config.config.GPT2Config object at 0x72c7df139990>


In [5]:
config.vocab_size

50257